<a href="https://colab.research.google.com/github/Riccardo-Venturi/Tesi_Script_Colab/blob/main/EstrattoreFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Riccardo VEnturi Cagliari 2025
#Questa è una funzione che puoi scrivere e testare in PyCharm, VSCode, o qualsiasi altro editor. Non ha bisogno di GPU o di Colab. Prende una maschera e sputa fuori numeri. Semplice, pulito, gratificante.
import cv2
import numpy as np
from pathlib import Path


def extract_features_from_mask(mask_path: Path):
    """
    Carica una maschera di predizione e calcola un set di feature quantitative.

    Args:
        mask_path (Path): Il percorso del file della maschera (es. "..._pred.png").

    Returns:
        dict: Un dizionario contenente le feature calcolate, o None se la maschera è invalida.
    """
    # --- 1. CARICAMENTO E PREPARAZIONE ---
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Errore: impossibile leggere la maschera {mask_path}")
        return None

    # Isola le singole classi in maschere binarie separate (0 o 1)
    foro_mask = (mask == 1).astype(np.uint8)
    danno_mask = (mask == 2).astype(np.uint8)

    # Inizializza il dizionario delle feature con valori di default
    features = {
        # Feature del Foro
        'area_foro_px': 0,
        'diametro_foro_px': 0,

        # Feature del Danno
        'area_danno_totale_px': 0,
        'numero_blob_danno': 0,
        'rapporto_danno_foro': 0.0,  # Metrica chiave, adimensionale

        # Feature di Forma del Danno (aggregate)
        'eccentricita_media_danno': 0.0,
        'solidita_media_danno': 0.0,
        'area_danno_piu_grande': 0.0,
    }

    # --- 2. ANALISI DEL FORO ---
    contours_foro, _ = cv2.findContours(foro_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours_foro:
        # Assumiamo che il contorno più grande sia il foro
        main_foro_contour = max(contours_foro, key=cv2.contourArea)
        area_foro = cv2.contourArea(main_foro_contour)
        if area_foro > 0:
            features['area_foro_px'] = area_foro
            # Diametro di un cerchio con la stessa area
            features['diametro_foro_px'] = np.sqrt(4 * area_foro / np.pi)

    # --- 3. ANALISI DEL DANNO (BLOB PER BLOB) ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(danno_mask, connectivity=8)

    # num_labels include lo sfondo (label 0), quindi il numero di blob è num_labels - 1
    num_blob_danno = num_labels - 1
    features['numero_blob_danno'] = num_blob_danno

    if num_blob_danno > 0:
        # Estrai le aree di tutti i blob di danno (escluso lo sfondo)
        aree_danno = stats[1:, cv2.CC_STAT_AREA]
        features['area_danno_totale_px'] = int(np.sum(aree_danno))
        features['area_danno_piu_grande'] = int(np.max(aree_danno))

        # Calcola le feature di forma per ogni blob e fanne la media
        eccentricities = []
        solidities = []
        for i in range(1, num_labels):
            blob_mask = (labels == i).astype(np.uint8)
            contours_danno, _ = cv2.findContours(blob_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not contours_danno: continue

            cnt = contours_danno[0]
            # Solidità: rapporto tra area del contorno e area del suo inviluppo convesso
            area_cnt = cv2.contourArea(cnt)
            hull = cv2.convexHull(cnt)
            hull_area = cv2.contourArea(hull)
            if hull_area > 0:
                solidities.append(area_cnt / hull_area)

            # Eccentricità: richiede di fittare un'ellisse
            if len(cnt) >= 5:  # fitEllipse richiede almeno 5 punti
                _, (MA, ma), _ = cv2.fitEllipse(cnt)
                eccentricities.append(np.sqrt(1 - (ma / MA) ** 2))

        if eccentricities:
            features['eccentricita_media_danno'] = np.mean(eccentricities)
        if solidities:
            features['solidita_media_danno'] = np.mean(solidities)

    # --- 4. CALCOLO FEATURE COMBINATE ---
    if features['area_foro_px'] > 0:
        features['rapporto_danno_foro'] = features['area_danno_totale_px'] / features['area_foro_px']

    return features
# --- ESEMPIO DI UTILIZZO ---
if __name__ == '__main__':
    # Crea una maschera finta per il test (o usa una delle tue maschere predette)
    # Assicurati che il percorso sia corretto
    test_mask_path = Path("/home/ricc/Downloads/MAsk_patches/H064_h064_Carbon Textile 1A.png") # Esempio

    if test_mask_path.exists():
        extracted_features = extract_features_from_mask(test_mask_path)
        if extracted_features:
            print(f"--- Feature estratte da {test_mask_path.name} ---")
            for key, value in extracted_features.items():
                print(f"{key:<30}: {value:.4f}")
    else:
        print(f"File di test non trovato: {test_mask_path}. Impossibile eseguire l'esempio.")
